## CNN: Fine-Tunning com Wisard

### Imports

In [7]:
import wisardpkg as wp
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from modules.termometro import codificador_termometro

### Carregando features e labels do modelo pré-treinado

In [8]:
features = np.load('features.npy', mmap_mode='r')
labels = np.load('labels.npy', mmap_mode='r')
print(features.shape)

(10000, 512)


### Binarização dos Atributos para o WiSARD usando o Termômetro

In [9]:
n_bits = 8
X_encoded_parts = []

# Aplica termômetro para cada feature
for i in range(features.shape[1]):
    coluna = features[:, i]
    coded = codificador_termometro(coluna, n_bits)
    X_encoded_parts.append(coded)

# Concatena todas as colunas codificadas horizontalmente
X_final_binary = np.hstack(X_encoded_parts).astype(np.uint8)


### Dividindo os Dados em Treino e Teste

In [10]:
# Se os rótulos não estiverem em string ainda:
y_str = labels.astype(str).tolist()

print(f"\nPrimeiros 5 rótulos de y (como strings): {y_str[:5]}")

X_encoded_uint8 = X_final_binary.astype(np.uint8)

print(f"\nDimensão de X após todas as codificações: {X_encoded_uint8.shape}")
print(f"Tipo dos elementos de X_final_binary: {X_encoded_uint8.dtype}")
print("Primeiras 2 linhas de X_final_binary:\n", X_encoded_uint8[:2])



Primeiros 5 rótulos de y (como strings): ['0', '1', '1', '0', '0']

Dimensão de X após todas as codificações: (10000, 4096)
Tipo dos elementos de X_final_binary: uint8
Primeiras 2 linhas de X_final_binary:
 [[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X_final_binary, y_str, test_size=0.3, random_state=42, stratify=y_str
)

print(f"Tamanho do conjunto de treino (X_train): {X_train.shape[0]} amostras, {X_train.shape[1]} bits")
print(f"Tamanho do conjunto de teste (X_test): {X_test.shape[0]} amostras, {X_test.shape[1]} bits")
print(f"Tipo dos elementos de X_train: {X_train.dtype}")

Tamanho do conjunto de treino (X_train): 7000 amostras, 4096 bits
Tamanho do conjunto de teste (X_test): 3000 amostras, 4096 bits
Tipo dos elementos de X_train: uint8


### Treinamento do Modelo WiSARD

In [12]:
addressSize = 8
wsd = wp.Wisard(addressSize, ignoreZero=False, verbose=True)

print(f"WiSARD inicializado com addressSize={addressSize}")
wsd.train(X_train, y_train)

predictions_str = wsd.classify(X_test)

y_pred_np = np.array(predictions_str)
y_test_np = np.array(y_test) 


WiSARD inicializado com addressSize=8


### Resultados e Conclusões

O modelo da WiSARD obteve melhores resultados se comparado ao de HDC mas ainda inferior ao da CNN original. No caso da WiSARD no entanto foi possível realizar a execução com 10000 amostras, talvez isso tenha influênciado no seu melhor resultado. No entanto, também houveram limitações por conta da máquina que estava executando o modelo, por conta da quantidade de memória RAM disponível (8 Gbytes)

In [14]:
print(classification_report(
    y_test_np, y_pred_np,
    labels=np.unique(y_test)
))

              precision    recall  f1-score   support

           0       0.80      0.91      0.85      1478
           1       0.90      0.79      0.84      1522

    accuracy                           0.84      3000
   macro avg       0.85      0.85      0.84      3000
weighted avg       0.85      0.84      0.84      3000

